# 5. Searching KItems with the SDK

In this tutorial we see how to search existing Kitems

### 5.1. Setting up
Before you run this tutorial: make sure to have access to a DSMS-instance of your interest, alongwith with installation of this package and have establised access to the DSMS through DSMS-SDK (refer to [Connecting to DSMS](../dsms_sdk.md#connecting-to-dsms))


Now let us import the needed classes and functions for this tutorial.

In [14]:
from dsms import DSMS, KItem

Now source the environmental variables from an `.env` file and start the DSMS-session.

In [15]:
dsms = DSMS(host_url="https://bue.materials-data.space", env=".env")

### 5.2. Searching for KItems

In this section, we would like to search for specfic KItems we created in the DSMS.

For this purpose, we will firstly create some KItems and apply the `search`-method on the `DSMS`-object later on in order to find them again in the DSMS.

We also want to demonstrate here, that we can link KItems to each other in order to find e.g. a related item of type `DatasetCatalog`. For this strategy, we are using the `linked_kitems`- attribute and the `id` of the item which we would like to link.

The procedure looks like this:

In [16]:
item1 = KItem(
    name="Machine-1",
    ktype_id=dsms.ktypes.TestingMachine,
    annotations=["https://w3id.org/steel/ProcessOntology/TestingMachine"],
    custom_properties={"Producer": "TestingLab GmBH",
                       "Room Number": "A404",
                       "Description": "Bending Test Machine"
                       }
)

item2 = KItem(
    name="Machine-2",
    ktype_id=dsms.ktypes.TestingMachine,
    annotations=["https://w3id.org/steel/ProcessOntology/TestingMachine"],
    custom_properties={"Producer": "StressStrain GmBH",
                       "Room Number": "B500",
                       "Description": "Compression Test Machine"
                       }
)

item3 = KItem(
    name="Specimen-1", 
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item1],
    annotations=["https://w3id.org/steel/ProcessOntology/TestPiece"],
    custom_properties={"Geometry": "Cylindrical 150mm x 20mm x 40mm",
                       "Material": "Concrete",
                       "Project ID": "ConstructionProject2024"
    }

)
item4 = KItem(
    name="Specimen-2",
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item2],
    annotations=["https://w3id.org/steel/ProcessOntology/TestPiece"],
    custom_properties={"Geometry": "Rectangular 200mm x 30mm x 20mm",
                       "Material": "Metal",
                       "Project ID": "MetalBlenders2024"
                       }
)

item5 = KItem(
    name="Research Institute ABC",
    ktype_id=dsms.ktypes.Organization,
    linked_kitems=[item1,item2],
    annotations=["www.researchBACiri.org/foo"],
)

dsms.commit()


<b> <p style="color:red;">Note : Here in this tutorial, we use dsms.search with `limit=1` to maintain readability but the user can adjust the variable `limit` as per requirement. </p>    </b> 


Now, we are apply to search for e.g. kitems of type `TestingMachine`:

In [17]:
for result in dsms.search(ktypes=[dsms.ktypes.TestingMachine], limit=1):
    print(result.hit)
    print("\n")

KItem(

	name = Machine-1, 

	id = d1524272-5f34-454b-bb61-7b0ab3a825a7, 

	ktype_id = testing-machine, 

	in_backend = True, 

	slug = machine-1-d1524272, 

	annotations = [], 

	attachments = [], 

	linked_kitems = [
		
			id: ba1a7871-ff75-465f-8625-a7a0202edca4
			name: Specimen-1
			slug: specimen-1-ba1a7871
			ktype_id: specimen
			summary: None
			avatar_exists: False
			annotations: []
			linked_kitems: [{
			id: d1524272-5f34-454b-bb61-7b0ab3a825a7
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'sampletype': None, 'sampleinformation': None, 'sampleproductionprocess': None, 'Geometry': 'Cylindrical 150mm x 20mm x 40mm', 'Material': 'Concrete', 'Project ID': 'ConstructionProject2024'}
			created_at: 2024-08-23T17:15:46.876917
			updated_at: 2024-08-23T17:15:46.876917
		, 
		
			id: 2335f57a-efc2-47ce-83b3-866cdeb6d30d
			name: R

... and for all of type `Organization` and `DatasetCatalog`:

In [18]:
for result in dsms.search(ktypes=[dsms.ktypes.Organization, dsms.ktypes.DatasetCatalog], limit=1):
    print(result.hit)
    print("fuzziness: ", result.fuzzy)
    print("\n")
    

KItem(

	name = Research Institute ABC, 

	id = 2335f57a-efc2-47ce-83b3-866cdeb6d30d, 

	ktype_id = organization, 

	in_backend = True, 

	slug = researchinstituteabc-2335f57a, 

	annotations = [
		{
			iri: www.researchBACiri.org/foo,
			name: research ABC Institute,
			namespace: research,
			description: None
		}
	], 

	attachments = [], 

	linked_kitems = [
		
			id: d1524272-5f34-454b-bb61-7b0ab3a825a7
			name: Machine-1
			slug: machine-1-d1524272
			ktype_id: testing-machine
			summary: None
			avatar_exists: False
			annotations: []
			linked_kitems: [{
			id: ba1a7871-ff75-465f-8625-a7a0202edca4
		}, {
			id: 2335f57a-efc2-47ce-83b3-866cdeb6d30d
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'Producer': 'TestingLab GmBH', 'Room Number': 'A404', 'Description': 'Bending Test Machine'}
			created_at: 2024-08-23T17:15:45.431622
		

... or for all of type `Dataset` with `Specimen-1` in the name:

In [19]:
for result in dsms.search(query="Specimen-1", ktypes=[dsms.ktypes.Dataset], limit=1):
    print(result.hit)
    print("\n")

... and for all of type `Organization` with the annotation `www.researchBACiri.org/foo`:

In [20]:
for result in dsms.search(
          ktypes=[dsms.ktypes.Organization], annotations=["www.researchBACiri.org/foo"], limit=1
    ):
    print(result.hit)
    print("\n")

KItem(

	name = Research Institute ABC, 

	id = 2335f57a-efc2-47ce-83b3-866cdeb6d30d, 

	ktype_id = organization, 

	in_backend = True, 

	slug = researchinstituteabc-2335f57a, 

	annotations = [
		{
			iri: www.researchBACiri.org/foo,
			name: research ABC Institute,
			namespace: research,
			description: None
		}
	], 

	attachments = [], 

	linked_kitems = [
		
			id: d1524272-5f34-454b-bb61-7b0ab3a825a7
			name: Machine-1
			slug: machine-1-d1524272
			ktype_id: testing-machine
			summary: None
			avatar_exists: False
			annotations: []
			linked_kitems: [{
			id: ba1a7871-ff75-465f-8625-a7a0202edca4
		}, {
			id: 2335f57a-efc2-47ce-83b3-866cdeb6d30d
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'Producer': 'TestingLab GmBH', 'Room Number': 'A404', 'Description': 'Bending Test Machine'}
			created_at: 2024-08-23T17:15:45.431622
		

## 5.3. Fetching linked KItems from a KItem

In the beginning under **5.1** we created some kitems and linked each other. Now we want to fetch the linked kitems and display them to the user. For this we use the `linked_kitems` attribute.

In [21]:
item5.linked_kitems

[
		
			id: be36f80f-7de9-43f4-86ca-d1175be14d4f
			name: Machine-1
			slug: machine-1-be36f80f
			ktype_id: testing-machine
			summary: None
			avatar_exists: False
			annotations: [{
			iri: https://w3id.org/steel/ProcessOntology/TestingMachine,
			name: TestingMachine,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}]
			linked_kitems: [{
			id: 0b72b38d-82fa-47cf-a2d2-7b9da797e7e9
		}, {
			id: 54c90f79-90b8-410c-8791-7ad0afd859e9
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'Producer': 'TestingLab GmBH', 'Room Number': 'A404', 'Description': 'Bending Test Machine'}
			created_at: 2024-08-23T17:41:31.074923
			updated_at: 2024-08-23T17:41:31.074923
		, 
		
			id: 6263e792-7cef-42c4-bb1e-6cfe3391f10e
			name: Machine-2
			slug: machine-2-6263e792
			ktype_id: testing-machine
			summary: None
			avatar

However, this linked KItem is not a "real" KItem. Due to performance reasons, we only display a slim representation of the linked KItem. Imagine if the linked KItem also has a linked KItem... this might lead to an infinite recursion. We can fetch the "real" KItem by using the `fetch` method of the linked KItem:


In [22]:
item5.linked_kitems[0].fetch()

KItem(

	name = Machine-1, 

	id = be36f80f-7de9-43f4-86ca-d1175be14d4f, 

	ktype_id = testing-machine, 

	in_backend = True, 

	slug = machine-1-be36f80f, 

	annotations = [
		{
			iri: https://w3id.org/steel/ProcessOntology/TestingMachine,
			name: TestingMachine,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}
	], 

	attachments = [], 

	linked_kitems = [
		
			id: 0b72b38d-82fa-47cf-a2d2-7b9da797e7e9
			name: Specimen-1
			slug: specimen-1-0b72b38d
			ktype_id: specimen
			summary: None
			avatar_exists: False
			annotations: [{
			iri: https://w3id.org/steel/ProcessOntology/TestPiece,
			name: TestPiece,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}]
			linked_kitems: [{
			id: be36f80f-7de9-43f4-86ca-d1175be14d4f
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'sample

We can also get the linked KItems grouped by annotation:

In [23]:
item5.linked_kitems.by_annotation

{'https://w3id.org/steel/ProcessOntology/TestingMachine': [
  			id: be36f80f-7de9-43f4-86ca-d1175be14d4f
  			name: Machine-1
  			slug: machine-1-be36f80f
  			ktype_id: testing-machine
  			summary: None
  			avatar_exists: False
  			annotations: [{
  			iri: https://w3id.org/steel/ProcessOntology/TestingMachine,
  			name: TestingMachine,
  			namespace: https://w3id.org/steel/ProcessOntology,
  			description: None
  		}]
  			linked_kitems: [{
  			id: 0b72b38d-82fa-47cf-a2d2-7b9da797e7e9
  		}, {
  			id: 54c90f79-90b8-410c-8791-7ad0afd859e9
  		}]
  			external_links: []
  			contacts: []
  			authors: [{
  			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
  		}]
  			linked_affiliations: []
  			attachments: []
  			user_groups: []
  			custom_properties: {'Producer': 'TestingLab GmBH', 'Room Number': 'A404', 'Description': 'Bending Test Machine'}
  			created_at: 2024-08-23T17:41:31.074923
  			updated_at: 2024-08-23T17:41:31.074923
  		,
  
  			id: 6263e792-7cef-42c4-bb1e-6

Again, we are able to fetch the "real" KItem by using the `fetch` method of the linked KItem:

In [24]:
item5.linked_kitems.by_annotation["https://w3id.org/steel/ProcessOntology/TestingMachine"][0].fetch()

KItem(

	name = Machine-1, 

	id = be36f80f-7de9-43f4-86ca-d1175be14d4f, 

	ktype_id = testing-machine, 

	in_backend = True, 

	slug = machine-1-be36f80f, 

	annotations = [
		{
			iri: https://w3id.org/steel/ProcessOntology/TestingMachine,
			name: TestingMachine,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}
	], 

	attachments = [], 

	linked_kitems = [
		
			id: 0b72b38d-82fa-47cf-a2d2-7b9da797e7e9
			name: Specimen-1
			slug: specimen-1-0b72b38d
			ktype_id: specimen
			summary: None
			avatar_exists: False
			annotations: [{
			iri: https://w3id.org/steel/ProcessOntology/TestPiece,
			name: TestPiece,
			namespace: https://w3id.org/steel/ProcessOntology,
			description: None
		}]
			linked_kitems: [{
			id: be36f80f-7de9-43f4-86ca-d1175be14d4f
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'sample

Clean up the DSMS from the tutortial:

In [25]:
del dsms[item1]
del dsms[item2]
del dsms[item3]
del dsms[item4]
del dsms[item5]

dsms.commit()